In [1]:
# Import CWI modules and point machine path to temporary path for using the model
import sys
sys.path.insert(0, './sequence-labeler-master')
from complex_labeller import Complexity_labeller

model_path = './cwi_seq.model'
temp_path = './temp_file.txt'

model = Complexity_labeller(model_path, temp_path)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` 

In [3]:
# import NLP modules and libraries
import spacy
import scispacy
from scispacy.linking import EntityLinker
nlp_med = spacy.load("en_ner_bc5cdr_md")
nlp_med.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

import readability_v_ks
nlp_read = readability_v_ks.spacy.load("en_core_web_sm")
nlp_read.add_pipe('readability')

#from nltk.corpus import wordnet as wn

In [5]:
# Import TFIDF tools
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Create dataframe for umls semantic tree for least specific hypernyms - most complexity reduction

columns = ['root_hypernym','TUI_ID','tree_level']
semantic_df = pd.read_csv(
    "modified_umls_semantic_type_tree.tsv",
    sep="\t",
    names=columns
)
semantic_wordphrases = semantic_df['root_hypernym'].values.tolist()
semantic_wordphrases = [x.lower() for x in semantic_wordphrases]
print(semantic_df)
print(semantic_wordphrases)

                root_hypernym TUI_ID  tree_level
0                       event   T051         1.0
1                    activity   T052         2.0
2                    behavior   T053         3.0
3                    behavior   T054         4.0
4                    behavior   T055         4.0
..                        ...    ...         ...
122           group-attribute   T102         3.0
123      intellectual-product   T170         3.0
124                regulation   T089         4.0
125            classification   T185         4.0
126                  language   T171         3.0

[127 rows x 3 columns]
['event', '  activity', '    behavior', '      behavior', '      behavior', '    activity', '    work-activity', '      healthcare-activity', '        lab-procedure', '        test-procedure', '        medical-procedure', '      research', '        molecular-research', '      regulatory-act', '      educational-act', '    machine-process', '  process', '    injury', '    human-effect',

In [7]:
# Define the TFIDF search engine
def tfidf_search(query, list_to_search):
    # Get tf-idf matrix using fit_transform function
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(list_to_search) # Store tf-idf representations of all docs
    query_vec = vectorizer.transform([query])  # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
    results = cosine_similarity(X, query_vec).reshape((-1,))  # Op -- (n_docs,1) -- Cosine Sim with each doc
    print("#####################################",query,"#####################################")
    print(X.shape) # (Number of wordphrases, Number of unique words)
    print("###################################################################################")
    tfidif_results = results.argsort()[-1:][::-1]
    return tfidif_results

    #for i in results.argsort()[-1:][::-1]:
        #print(df.iloc[i, 0], "--", df.iloc[i, 1],"--", df.iloc[i, 3])

In [37]:
# Define the automatic translator function
def automatic_translation(document, semantic_df, runGinger):
    # Converting sentence/document
    test_document = document
    new_document = test_document
    Complexity_labeller.convert_format_string(model, test_document)
    dataframe = Complexity_labeller.get_dataframe(model)
    # Access binary labeling information from the dataframe format:
    cw_list = list(zip(dataframe['sentences'].values[0], dataframe['labels'].values[0], dataframe['probs'].values[0]))
    # get_bin_labels returns the binary complexity labels for the input
    # bin_label_list = Complexity_labeller.get_bin_labels(model)
    # The `get_prob_labels` method returns the probability of each token belonging to the complex class.
    # prob_label_list = Complexity_labeller.get_prob_labels(model)
    doc_med = nlp_med(test_document)
    linker = nlp_med.get_pipe("scispacy_linker")

    for token in doc_med.ents:
        token_string = str(token)
        token_string_list = token_string.split(' ')
        tui = ''

        try:
            umls_data = linker.kb.cui_to_entity[token._.kb_ents[0][0]]
            umls_data = str(umls_data).split('\n')
            tui = umls_data[2]

        except:
            pass
            #print('no umls entity found for token')

        try:
            tui_split = tui.split(' ')
            #tui_list = []
            #highest_level = 0
            #previous_tui = ''
            #previous_tui_word = ''
            #final_tui = ''
            #final_tui_word = ''
            for tui in tui_split:

                try:
                    tui = tui.strip(',')
                    tui_word_row = semantic_df.loc[semantic_df['TUI_ID'] == tui] # pandas sem df find row with matching tui
                    tui_word = tui_word_row.iloc[0,0] # tui_word is a single row 3 column df, select row 0 (only row), column 0 (word)
                    tui_tree_level = tui_word_row.iloc[0,2] # same as above but the tree level is in third cell, aka index 2, of each row
                    tui_word = str(tui_word).lstrip()
                    # pretty sure this whole code block is unecessary probably because the for tui loop ends on last tui which will be highest in tree
                    #if tui_tree_level > highest_level:
                        #highest_level = tui_tree_level
                        #final_tui_word = tui_word
                        #final_tui = tui

                    #elif tui_tree_level == highest_level:
                        #tui_int_current = int(tui.strip('T'))
                        #tui_int_previous = int(previous_tui.strip('T'))
                        #if tui_int_current >= tui_int_previous:
                            #final_tui = tui
                            #final_tui_word = tui_word
                        #elif tui_int_current < tui_int_previous:
                            #final_tui = previous_tui
                            #final_tui_word = previous_tui_word
                    #else:
                        #pass

                    previous_tui = tui
                    previous_tui_word = tui_word
                except:
                    pass

        except IndexError or UnboundLocalError:
            pass

        for i in cw_list:
            cw = i[0]
            cw_bin_complexity = i[1]
            #cw_prob_complexity_1 = i[2]
            #cw_prob_complexity_2 = str(cw_prob_complexity_1[1])
            #cw_prob_complexity_3 = float(cw_prob_complexity_2)

            if cw in token_string_list and cw_bin_complexity == 1:
                #token2 = wn.synsets(cw)
                try:
                    #hypernym = token2[0].hypernyms()
                    #hypernym = str(hypernym).split("'")[1].split(".")[0]
                    if tui_word != '':
                        new_document = new_document.replace(str(token), tui_word.lower())

                except:
                    pass
    new_document = new_document.lower()
    new_document = runGinger(new_document)
    doc_med = nlp_read(document)
    og_fk_score = str(doc_med._.flesch_kincaid_reading_ease)
    og_fk_grade = str(doc_med._.flesch_kincaid_grade_level)
    og_dc_score = str(doc_med._.dale_chall)
    og_smog_score = str(doc_med._.smog)
    og_cl_score = str(doc_med._.coleman_liau_index)
    og_ar_index = str(doc_med._.automated_readability_index)
    og_forcast = str(doc_med._.forcast)

    #new_document = new_document['result'].replace('_',' ')
    doc_new = nlp_read(new_document)
    new_fk_score = str(doc_new._.flesch_kincaid_reading_ease)
    new_fk_grade = str(doc_new._.flesch_kincaid_grade_level)
    new_dc_score = str(doc_new._.dale_chall)
    #new_smog_score = str(doc_new._.smog)
    new_cl_score = str(doc_new._.coleman_liau_index)
    new_ar_index = str(doc_new._.automated_readability_index)
    #new_forcast = str(doc_new._.forcast)

    #if new_fk_score <= 8 and new_fk_grade > og_fk_score: # I know this seems backwards but due to some weird shit with word embeddings the matrices I think are like run backwards, so it switches the axes, if you put in right order by variable name, the names will be right but the numbers will be wrong.
    with open('test_5_output_all_notes.txt', 'a+') as f:
        f.write(test_document+'\n')
        f.write(new_document+'\n')
        f.write('fk_score'+'\t\t'+og_fk_score+'\t\t'+new_fk_score+'\n')
        f.write('fk_grade'+'\t\t'+og_fk_grade+'\t\t'+new_fk_grade+'\n')
        f.write('dc_score'+'\t\t'+og_dc_score+'\t\t'+new_dc_score+'\n')
        #f.write('smog'+'\t\t'+og_smog_score+'\t\t'+new_smog_score+'\n')
        f.write('cl_score'+'\t\t'+og_cl_score+'\t\t'+new_cl_score+'\n')
        f.write('ar_index'+'\t\t'+og_ar_index+'\t\t'+new_ar_index+'\n')
        #f.write('forcast'+'\t\t'+og_forcast+'\t\t'+new_forcast+'\n')
        f.write('\n')


In [38]:
#from gingerit.gingerit import GingerIt
#parser = GingerIt()

from gingerit.gingerit import GingerIt # pip install gingerit
#import pandas as pd
import pysbd, re # pip install pysbd
segmentor = pysbd.Segmenter(language="en", clean=False)
subsegment_re = r'[^;:\n•]+[;,:\n•]?\s*'

def runGinger(par):
    fixed = []
    for sentence in segmentor.segment(par):
        if len(sentence) < 300:
            fixed.append(GingerIt().parse(sentence)['result'])
        else:
            subsegments = re.findall(subsegment_re, sentence)
            if len(subsegments) == 1 or any(len(v) < 300 for v in subsegments):
                # print(f'Skipped: {sentence}') // No grammar check possible
                fixed.append(sentence)
            else:
                res = []
                for s in subsegments:
                    res.append(GingerIt().parse(s)['result'])
                fixed.append("".join(res))
    return " ".join(fixed)

#data['jd'] = data['Job Description'].apply(lambda x: runGinger(x))

In [39]:
#for line in open("/home/karl/PycharmProjects/doctorlingo/UMLS/MRDEF.RRF", 'r'):
#for line in open("test_notes.txt", 'r'):
#for i,line in enumerate(sent_list):
for line in open("dataset_parsed_all_mimic_test_notes.txt", 'r'):
    print(line)
    document = str(line.strip('\n'))
    automatic_translation(document, semantic_df, runGinger)
    #source = str(line.split('|')[4])
    #if source == 'MSH':
    #    document = str(line.split('|')[5])
    #    automatic_translation(document.lower(), semantic_df)
    #else:
    #    pass

O: 58 YEAR OLD FEMALE ADMITTED IN TRANSFER FROM CALVERT HOSPITAL FOR MENTAL STATUS CHANGES POST FALL AT HOME AND CONTINUED HYPOTENSION AT CALVERT HOSPITAL REQUIRING DOPAMINE; PMH: CAD, S/P MI 1992; LCX PTCA; 3V CABG WITH MVR; CMP; AFIB- AV NODE ABLATION; PERM PACER- DDD MODE; PULM HTN; PVD; NIDDM; HPI: 2 WEEK HISTORY LEG WEAKNESS; 7/22 FOUND BY HUSBAND ON FLOOR- AWAKE, BUT MENTAL STATUS CHANGES; TO CALVERT HOSPITAL ER- TO THEIR ICU; HEAD CT- NEG FOR BLEED; VQ SCAN- NEG FOR PE; ECHO- GLOBAL HYPOKINESIS; EF EST 20%; R/O FOR MI; DIGOXIN TOXIC WITH HYPERKALEMIA- KAYEXALATE, DEXTROSE, INSULIN; RENAL INSUFFICIENCY- BUN 54, CR 2.8; INR 7 ( ON COUMADIN AT HOME); 7/23 AT CALVERT- 2 FFP, 2 UNITS PRBC, VITAMIN K; REFERRED TO GH.  ARRIVED IN TRANSFER APPROX. 2130; IN NO MAJOR DISTRESS; DOPAMINE TAPER, THEN DC; NS FLUID BOLUS GIVEN WITH IMPROVEMENT IN BP RANGE; SEE FLOW SHEET SECTION FOR CLINICAL INFORMATION; A: NO HEMODYNAMIC COMPROMISE SINCE TRANSFER; TOLERATING DOPAMINE DC; P: TREND BP RANGE; OB

/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


O: BUN NOW 50; CR 2.1- URINE OUTPUT IMPROVING SLIGHTLY POST FLUID CHALLENGE; K 3.9; A: CONTINUED RENAL INSUFFICIENCY/ RENAL FAILURE. P: FOR REPEAT BUN,CR,K; CONTINUE I&O.

O: INR 2.0; PT 17.5; PTT 32.3; IV HEPARIN BEGUN AT 1100 UNITS POST BOLUS; 5 MG COUMADIN GIVEN. A: ANTICOAGULATION BEGUN FOR HISTORY OF MECHANICAK VALVE, HISTORY OF AFIB; P: MAINTAIN IV HEPARIN PER PROTOCOL; FOR AM PT, PTT.

58 YR OLD ADMITTED TO CALVERT HOSPIATAL P FALL C MENTAL STATUS CHANGES REQUIRING DOPAMINE . CT SCAN NEG . R/O MI , VQ SCAN NEG, HYPERKALEMIA RX C KAYEXALATE  PMH MI 92, 3 VCABG C MVR,ON COUMADIN, EF 20% ,AV NODE ABLATION FOR A FIB, DDD PERM PACER , PULM HYPERTENSION ,PVD, NIDDM ,2 WKS LEG WEAKNESS   TRANS TO GH 7/23 .DOPAMINE WEANED TO OFF . PLAN TO RESTART CARDIAC MEDS ,ADVANCE ACTIVITY, TRANS TO FLOOR.   CV PACED R. NO ECTOPY . BP STABLE OFF DOPAMINE,OOB TOL WELL . DIG RESTARTED . HEPARIN DECREASED TO 900 U FOR PTT 119 . COUMADIN HAS BEEN RESTARTED . LEGS EDEMATOUS  RESP RALES IN BASES ,EXERTION

KeyError: 'Corrections'

# For v2:
## Create to the point! - a module that takes an EHR note finds bottom line [diagnoses] and [plan] for such, tells patient they have [condition], which is a [definition], that they must do [plan] for.
Do this with NER based rules. If entity == label_ then return that variable.

Include a section called "Why your doctor thinks you have this." Then create NER based rules that recognize subjective, physical exam findings etc.

Thought of idea of all permutations and combinations, but again scorers really just grade based on number words, word length and sentence length.

## Med7 to resolve medicines
Need to figure out how to use two versions of spacy
Can a python script switch environments?

## Create hypernymy NER to make a training set, this will allow us to train a machine learning model that automatically subs hypernyms

## Steal the nature paper methods to resolve abbreviations or figure out how to resolve them yourself